In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch.utils.data as data_utils
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch
import torch.nn.functional as F

In [4]:
RANDOM_SEED = 77
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
df = pd.read_csv('/Users/wbchoi/PycharmProjects/ai_projects/C_ITS/prediction_modeling/full_data_kma_hours_lightgbm_feature_extract.csv')
data_path = '/Users/wbchoi/PycharmProjects/ai_projects/C_ITS/prediction_modeling/full_data_kma_hours_lightgbm_feature_extract.csv'

In [6]:
df.head()

,SUB_YN,ROAD_TYPE,REST_VEH,MAX_SPD,LANES,CW_YN,BUS_YN,A3_ROADTYPE_1_YN,A1_LANE_04_YN,A1_BARR_03_YN,...,A1_BARR_03_CNT,A1_BARR_02_CNT,topis_22_07,pop_22_07,WS,WD,TEMP,RF,HM,SAGO_YN
0,0,0,0,60.0,5.0,1,1,0,0,0,...,0.0,0.0,0.250954,0.024952,0.064220,0.943889,0.419935,0.0,0.727227,0
1,0,0,0,60.0,5.0,1,1,0,0,0,...,0.0,0.0,0.250954,0.024952,0.146789,0.911667,0.418301,0.0,0.707207,0
2,0,0,0,60.0,5.0,1,1,0,0,0,...,0.0,0.0,0.250954,0.024952,0.100917,0.956667,0.410131,0.0,0.755255,0
3,0,0,0,60.0,5.0,1,1,0,0,0,...,0.0,0.0,0.250954,0.024952,0.100917,0.963611,0.410131,0.0,0.764765,0
4,0,0,0,60.0,5.0,1,1,0,0,0,...,0.0,0.0,0.250954,0.024952,0.128440,0.973333,0.405229,0.0,0.775275,0


In [7]:
df.shape

(15797629, 37)

In [8]:
# df0 = df.query('SAGO_YN == 0').sample(20000)
# df1 = df.query('SAGO_YN == 1').sample(400)
# df = pd.concat([df0, df1])

In [9]:
# df.shape

In [22]:
batch_size = 1024
lr = 10e-5

D_in = x_train.shape[1]
H = 18
H2 = 9

In [23]:
x_train, x_test, y_train, y_test = train_test_split(df.drop(labels=['SAGO_YN'], axis = 1) , 
                                                    df['SAGO_YN'], test_size=0.2, random_state=42)
print(x_train.shape, 'train samples')
print(x_test.shape, 'test samples')

(12638103, 36) train samples
(3159526, 36) test samples


In [24]:
train_loader = data_utils.DataLoader(x_train,batch_size=batch_size,shuffle=True)
test_loader = data_utils.DataLoader(x_test, batch_size=1, shuffle=False)

In [43]:
class VAE(nn.Module):
    def __init__(self,D_in, H, H2, latent_dim=4):
        
        # Encoder
        super(VAE,self).__init__()
        self.linear1=nn.Linear(D_in,H)
        self.lin_bn1 = nn.BatchNorm1d(num_features=H)
        self.linear2=nn.Linear(H,H2)
        self.lin_bn2 = nn.BatchNorm1d(num_features=H2)
        # self.linear3=nn.Linear(H2,H2)
        # self.lin_bn3 = nn.BatchNorm1d(num_features=H2)
        
        # Latent vectors mu and sigma
        self.fc1 = nn.Linear(H2, latent_dim)
        self.bn1 = nn.BatchNorm1d(num_features=latent_dim)
        self.fc21 = nn.Linear(latent_dim, latent_dim)
        self.fc22 = nn.Linear(latent_dim, latent_dim)

        # Sampling vector
        self.fc3 = nn.Linear(latent_dim, latent_dim)
        self.fc_bn3 = nn.BatchNorm1d(latent_dim)
        self.fc4 = nn.Linear(latent_dim, H2)
        self.fc_bn4 = nn.BatchNorm1d(H2)
        
        # Decoder
        # self.linear4=nn.Linear(H2,H2)
        # self.lin_bn4 = nn.BatchNorm1d(num_features=H2)
        self.linear5=nn.Linear(H2,H)
        self.lin_bn5 = nn.BatchNorm1d(num_features=H)
        self.linear6=nn.Linear(H,D_in)
        self.lin_bn6 = nn.BatchNorm1d(num_features=D_in)
        
        self.relu = nn.ReLU()
        
    def encode(self, x):
        lin1 = self.relu(self.lin_bn1(self.linear1(x)))
        lin2 = self.relu(self.lin_bn2(self.linear2(lin1)))
        # lin3 = self.relu(self.lin_bn3(self.linear3(lin2)))

        # fc1 = F.relu(self.bn1(self.fc1(lin3)))
        fc1 = F.relu(self.bn1(self.fc1(lin2)))

        r1 = self.fc21(fc1)
        r2 = self.fc22(fc1)
        
        return r1, r2
    
    def reparameterize(self, mu, logvar):
        if self.training:
            std = logvar.mul(0.5).exp_()
            eps = Variable(std.data.new(std.size()).normal_())
            return eps.mul(std).add_(mu)
        else:
            return mu
        
    def decode(self, z):
        fc3 = self.relu(self.fc_bn3(self.fc3(z)))
        fc4 = self.relu(self.fc_bn4(self.fc4(fc3)))

        lin4 = self.relu(self.lin_bn4(self.linear4(fc4)))
        lin5 = self.relu(self.lin_bn5(self.linear5(lin4)))
        return self.lin_bn6(self.linear6(lin5))

        
    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        # self.decode(z) ist später recon_batch, mu ist mu und logvar ist logvar
        return self.decode(z), mu, logvar

In [44]:
'''class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(x_train.shape[1], 18)
        self.fc21 = nn.Linear(18, 4)
        self.fc22 = nn.Linear(18, 4)
        self.fc3 = nn.Linear(4, 18)
        self.fc4 = nn.Linear(18, x_train.shape[1])

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        if self.training:
            std = torch.exp(0.5*logvar)
            eps = torch.randn_like(std)
            return eps.mul(std).add_(mu)
        else:
            return mu

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return F.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, x_train.shape[1]))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar'''

'class VAE(nn.Module):\n    def __init__(self):\n        super(VAE, self).__init__()\n\n        self.fc1 = nn.Linear(x_train.shape[1], 18)\n        self.fc21 = nn.Linear(18, 4)\n        self.fc22 = nn.Linear(18, 4)\n        self.fc3 = nn.Linear(4, 18)\n        self.fc4 = nn.Linear(18, x_train.shape[1])\n\n    def encode(self, x):\n        h1 = F.relu(self.fc1(x))\n        return self.fc21(h1), self.fc22(h1)\n\n    def reparameterize(self, mu, logvar):\n        if self.training:\n            std = torch.exp(0.5*logvar)\n            eps = torch.randn_like(std)\n            return eps.mul(std).add_(mu)\n        else:\n            return mu\n\n    def decode(self, z):\n        h3 = F.relu(self.fc3(z))\n        return F.sigmoid(self.fc4(h3))\n\n    def forward(self, x):\n        mu, logvar = self.encode(x.view(-1, x_train.shape[1]))\n        z = self.reparameterize(mu, logvar)\n        return self.decode(z), mu, logvar'

In [45]:
class loss_function(nn.Module):
    def __init__(self):
        super(loss_function, self).__init__()
        self.mse_loss = nn.MSELoss(reduction="sum")
    
    # x_recon ist der im forward im Model erstellte recon_batch, x ist der originale x Batch, mu ist mu und logvar ist logvar 
    def forward(self, x_recon, x, mu, logvar):
        loss_MSE = self.mse_loss(x_recon, x)
        loss_KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

        return loss_MSE + loss_KLD

In [46]:
# takes in a module and applies the specified weight initialization
def weights_init_uniform_rule(m):
    classname = m.__class__.__name__
    # for every Linear layer in a model..
    if classname.find('Linear') != -1:
        # get the number of the inputs
        n = m.in_features
        y = 1.0/np.sqrt(n)
        m.weight.data.uniform_(-y, y)
        m.bias.data.fill_(0)

In [47]:
model = VAE(D_in, H, H2).to(device)
model.apply(weights_init_uniform_rule)
# model = model.double()

optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [48]:
loss_mse = loss_function()

In [49]:
num_epochs = 3
val_losses = []
train_losses = []

In [50]:
def train(epoch):
    
    model.train()
    train_loss = 0
    
    for batch_idx, data in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_mse(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()

    if epoch % 1 == 0:        
        print('====> Epoch: {} Average loss: {:.4f}'.format(
            epoch, train_loss / len(train_loader.dataset)))
        train_losses.append(train_loss / len(train_loader.dataset))

In [51]:
for epoch in range(1, num_epochs + 1):
    train(epoch)

KeyError: 5965470